In [3]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)

from utils import load_hidden_representations_from_hdf5, read_templates_from_file, read_predictions_from_file

In [4]:
# params
log_dir = "/logfiles"
model = "bigscience-T0_3B"
module = "encoder"
task = "rte"

In [5]:
df = read_templates_from_file(f"/t0-analysis/prompts/{task}/all.csv")
# df = read_templates_from_file(f"/t0-analysis/prompts/{task}/fixed_prompt.csv")
# df = read_templates_from_file(f"/t0-analysis/prompts/{task}/fixed_target_yes_no.csv")
display(df)

,name,template,category,includes_targets,targets,target_ids,shuffle
0,gpt_3_yes_no_with_targets,{premise} Question: {hypothesis} Yes or No?,instructive,True,"▁Yes, ▁No, ▁yes, ▁no","0, 1, 0, 1",False
1,mnli_crowdsource_with_targets,"{premise} Using only the above description and what you know about the world, is ""{hypothesis}"" definitely correct? Yes or No?",instructive,True,"▁Yes, ▁No, ▁yes, ▁no","0, 1, 0, 1",False
2,based_on_previous_passage_with_targets,"{premise} Based on the previous passage, is it true that ""{hypothesis}""? Yes or No?",instructive,True,"▁Yes, ▁No, ▁yes, ▁no","0, 1, 0, 1",False
3,infer_with_targets,"Suppose {premise} Can we infer that ""{hypothesis}""? Yes or No?",instructive,True,"▁Yes, ▁No, ▁yes, ▁no","0, 1, 0, 1",False
4,follow_with_targets,Given that {premise} Does it follow that {hypothesis} Yes or No?,instructive,True,"▁Yes, ▁No, ▁yes, ▁no","0, 1, 0, 1",False
5,imply_with_targets,"{premise} Question: Does this imply that ""{hypothesis}""? Yes or No?",instructive,True,"▁Yes, ▁No, ▁yes, ▁no","0, 1, 0, 1",False
6,guaranteed_with_targets,"Given {premise} Is it guaranteed true that ""{hypothesis}""? Yes or No?",instructive,True,"▁Yes, ▁No, ▁yes, ▁no","0, 1, 0, 1",False
7,justified_with_targets,"{premise} Are we justified in saying that ""{hypothesis}""? Yes or No?",instructive,True,"▁Yes, ▁No, ▁yes, ▁no","0, 1, 0, 1",False
8,must_be_true_with_targets,"Given that {premise} Therefore, it must be true that ""{hypothesis}""? Yes or No?",instructive,True,"▁Yes, ▁No, ▁yes, ▁no","0, 1, 0, 1",False
9,should_assume_with_targets,"Given {premise} Should we assume that ""{hypothesis}"" is true? Yes or No?",instructive,True,"▁Yes, ▁No, ▁yes, ▁no","0, 1, 0, 1",False


In [6]:
# load prompted inputs and labels
df_inputs = {}
for _, row in df.iterrows():
    prompt = row['name']
    df_inputs[prompt] = pd.read_csv(f"/logfiles/{task}/{model}/{module}/{prompt}/prompted_samples.csv", sep='\t', index_col=0)


In [18]:
use_pattern = [
    "gpt_3_yes_no_with_targets",
    # "mnli_crowdsource_with_targets",
    # "based_on_previous_passage_with_targets",
    # "infer_with_targets",
    # "follow_with_targets",
    # "imply_with_targets",
    # "guaranteed_with_targets",
    # "justified_with_targets",
    "must_be_true_with_targets",
    # "should_assume_with_targets",
    # "gpt_3_true_false_with_targets",
    # "gpt_3_cat_dog_with_targets",
    # "gpt_3_cat_dog_with_targets_yes_no",
    # "gpt_3_yes_no_without_targets",
    # "words_appear_with_targets",
    # "similar_words_with_targets",
    # "start_with_the_with_targets",
    # "same_meaning_with_targets",
    # "paraphrase_with_targets",
    # "paraphrase_r_with_targets",
    # "summarize_with_targets",
    # "inflection_with_targets",
    # "null_pattern_with_targets",
    # "null_pattern_r_with_targets",
    # "null_pattern_without_targets",
    # "null_pattern_r_without_targets",
    "premise_only_with_targets",
    # "premise_only_without_targets",
    "hypothesis_only_with_targets",
    # "hypothesis_only_without_targets",
]

In [25]:
for sidx in range(20):
    for _, row in df.iterrows():
        prompt = row["name"]
        if prompt in use_pattern:
            row = df_inputs[prompt].iloc[sidx]
            print(f"{sidx}: {row['input']} ({prompt})")
            # print(sidx, row["input"], row["label"], row["prediction"])
    print("")
    


0: Dana Reeve, the widow of the actor Christopher Reeve, has died of lung cancer at age 44, according to the Christopher Reeve Foundation. Question: Christopher Reeve had an accident. Yes or No? (gpt_3_yes_no_with_targets)
0: Given that Dana Reeve, the widow of the actor Christopher Reeve, has died of lung cancer at age 44, according to the Christopher Reeve Foundation. Therefore, it must be true that "Christopher Reeve had an accident."? Yes or No? (must_be_true_with_targets)
0: Dana Reeve, the widow of the actor Christopher Reeve, has died of lung cancer at age 44, according to the Christopher Reeve Foundation.? Yes or No? (premise_only_with_targets)
0: Christopher Reeve had an accident.? Yes or No? (hypothesis_only_with_targets)

1: Yet, we now are discovering that antibiotics are losing their effectiveness against illness. Disease-causing bacteria are mutating faster than we can come up with new antibiotics to fight the new variations. Question: Bacteria is winning the war against 

In [24]:
# look at some "features"
for sidx in range(20):
    for _, row in df.iterrows():
        prompt = row["name"]
        if prompt in use_pattern:
            row = df_inputs[prompt].iloc[sidx]
            print(f"{sidx}: len={len(row['input'].split())}; label={row['label']}; prediction={row['prediction']}")
            # print(sidx, row["input"], row["label"], row["prediction"])
    print("")
    


0: len=32; label=not_entailment; prediction=not_entailment
0: len=39; label=not_entailment; prediction=not_entailment
0: len=26; label=not_entailment; prediction=not_entailment
0: len=8; label=not_entailment; prediction=not_entailment

1: len=42; label=entailment; prediction=not_entailment
1: len=49; label=entailment; prediction=not_entailment
1: len=34; label=entailment; prediction=not_entailment
1: len=10; label=entailment; prediction=not_entailment

2: len=141; label=not_entailment; prediction=not_entailment
2: len=148; label=not_entailment; prediction=not_entailment
2: len=130; label=not_entailment; prediction=not_entailment
2: len=13; label=not_entailment; prediction=not_entailment

3: len=112; label=not_entailment; prediction=not_entailment
3: len=119; label=not_entailment; prediction=not_entailment
3: len=102; label=not_entailment; prediction=not_entailment
3: len=12; label=not_entailment; prediction=not_entailment

4: len=39; label=entailment; prediction=not_entailment
4: len=4